# TAIL RISK FUND: OPTIONS (BHANSALI STRATEGY)

author: [@sparshsah](https://github.com/sparshsah)

ref: https://github.com/sparshsah/foggy-demo/blob/main/demo/finance/tail-risk-fund_options.ipynb

In [16]:
from __future__ import annotations

import dataclasses
import datetime
from typing import Final, Literal

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# https://github.com/sparshsah/foggy-statslib
import sys; sys.path.append("../../../foggy-statslib/")
import foggy_statslib.core as fsc
import foggy_statslib.fin as fsf

In [29]:
BUDGET_DOL: Final[float] = 100
START_DATE: Final[datetime.datetime] = datetime.datetime(2014, 7, 1)
IMPL_LAG: Final[int] = 3  # trading lag in days


def _calc_vol(r: pd.Series[float]) -> pd.Series[float]:
    """Hackily calculate realized vol. Assumes `freq="B"`."""
    return (
        (
            r
            # assert that ER_daily is close to zero
            .abs()
            # hackily remove influence of stock splits
            .clip(upper=0.20)
            **2
        )
        # a business quarter
        .rolling(window=65)
        .mean()
        **0.5
        # annualize
        * 261**0.5
    )


@dataclasses.dataclass(kw_only=True, frozen=True)
class StrategyConfig:
    """By default, buy puts quarterly at 20% OTM, selling iff value hits 2x initial price."""
    period_freq: Literal["M", "Q", "Y"] = "Q"
    oom_px: float = 0.20
    monetization_thresh: float = 2.0


class MarketDataAccess:
    """Module for getting data."""

    @staticmethod
    def _get_history_for_stock(ticker: str = "COF") -> pd.DataFrame:
        p = (
            pd.read_csv(
                f"data/yahoo-finance_NYSE-{ticker}_px.csv",
                index_col="Date",
                parse_dates=True,
            )
            .asfreq("D").ffill().asfreq("B")
            .loc[START_DATE:, "Close"].rename("price_close_unadj")
        )
        dtx = pd.date_range(
            start=p.index[0],
            end=p.index[-1],
            freq="D",
            name="date",
        )
        p = p
        r = p / p.shift() - 1
        
        return p

SyntaxError: '(' was never closed (2672001077.py, line 61)

In [31]:
x = MarketDataAccess._get_history_for_stock()

In [33]:
x.asfreq("D")

date
2014-07-01     83.019997
2014-07-02     83.599998
2014-07-03     84.949997
2014-07-04     84.949997
2014-07-05           NaN
                 ...    
2023-11-27    106.180000
2023-11-28    108.580002
2023-11-29    110.129997
2023-11-30    111.660004
2023-12-01    113.930000
Freq: D, Name: close_raw, Length: 3441, dtype: float64

In [ ]:
START_DATE